In [2]:
import torch


def reward(
    task_score: float,
    language_score: float,
    t_weight: float = 1.0,
    l_weight: float = 1.0,
) -> float:
    """
    Computes the harmonic weighted reward score based on task success and language aspect.

    This method calculates a reward by combining the task performance score 
    with a penalty based on the language aspect. The final score is normalized 
    to the range [0, 1].

    Parameters
    ----------
    task_score : float
        A score in the range [0, 1] indicating task performance.
    language_score : float
        A score in the range [0, 1] indicating adherence to language constraints. It equals in our cases often to 1 - penalisation_score, where penalisation_score is included between 0 and 1.
    t_weight : float, optional
        The weight assigned to the task score (default is 1.0).
    l_weight : float, optional
        The weight assigned to the language score penalty (default is 1.0).

    Returns
    -------
    float
        A normalized reward score in the range [0, 1].

    Notes
    -----
    - First implementation was using a weighted average score, which induced the following issue : a random agent would have a constant reward of 0.5, whereas , in pricing game, a failing calculation of bid additioned to using P1 information was leading to a reward of 0. 
    In math task example, saying random things without using equal sign was leading as well to a reward 0.5, where trying to solve the problem was inducing a worst reward. We hope to fix this problematic case with a weighted harmonic reward.
    - A higher `task_score` increases the reward.
    - A higher `language_score` decreases the reward.
    - A low task score or language score induces a low reward.
    - Custom reward functions can be implemented by overriding this method.
    """
    r = (t_weight + l_weight) * task_score * language_score / (t_weight * language_score + l_weight * task_score + torch.finfo(torch.float32).eps)

    return r

In [4]:
reward(0.6584, 0.1394)

0.23008509973893046

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

In [27]:
token_ids = tokenizer("""I need to understand the current situation. The supplier has disclosed that their material costs represent 42.371% of their total manufacturing cost per unit. Their latest component order shows $5.069 per unit in materials. Standard pricing policy sets final price at 2.223x total manufacturing cost.

So, the base cost is $5.069 per unit, and the total manufacturing cost is 100% of the base cost. The total manufacturing cost is $5.069 per unit.

The supplier has offered a 3.466% discount per unit.

Therefore, the price I will propose is 3.466% of the total manufacturing cost, which is $5.069 per unit.

So, my final answer is $5.069 * (1 - 3.466%) = $5.069 * 0.96534 = $5.069 - $0.01868 = $4.999 approximately.

But let's be precise.

Total manufacturing cost: $5.069

Discount: 3.466% of $5.069

Discount amount: 0.03466 * $5.069 = $0.1759

Therefore, my bid price: $5.069 - $0.1759 = $4.8931

So, my final answer is $4.8931
</think>""", add_special_tokens=False)["input_ids"]

In [28]:
len(token_ids)

329